# Oanda Demo Trading Notebook

#### Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import winsound
import collections

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

#### API Setup

Read from config file

In [3]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

#### Retrive Data

In [4]:
# Get timestamp of the price and segregate it
def get_date_time(resp):
    
    time_stamp = resp['time']
    date_val, full_time = time_stamp.split(sep = 'T')
    time_val, time_fraction = full_time.split(sep = '.')
    
    return(date_val, time_val, time_fraction)

In [5]:
# Get bid and ask prices
def get_prices(resp):
    
    bid_price = float(resp['bids'][0]['price'])    
    ask_price = float(resp['asks'][0]['price'])
    spread = ask_price - bid_price
    
    return(bid_price, ask_price, spread)

In [6]:
# Terminate connection
def terminate_connection():
    try:
        print('Connection Terminated')
        r.terminate(message = "maxrecs records received")
    except:
        pass

In [7]:
# Add prices into chunks for direction identification
def get_chunks(i, val):
    global chunk_size
    global small_list
    global size_flag
    global change_position
    
    if i < chunk_size: #Keep adding prices until we reach chunk size
        small_list.append(val)  
               
    if size_flag == 1:
        small_list.popleft()   #Remove the left most (first) price
        small_list.append(val) #Add current value to right most position
        
        change_position -= 1  #Price allocation in small list happens right to left so change position goes from chunk size to zero
        if change_position < 0: 
            change_position = chunk_size - 1 #If position reaches zero, it needs to be reset to chunk size - 1 (for index position)
    
    if len(small_list) == chunk_size: #check if small list has reached chunk size
        size_flag = 1  
    
    return(small_list)

In [8]:
# Add prices into chunks for direction identification
def get_chunks1(i, val):
    global chunk_size
    global small_list
    global size_flag
    global change_position
    
    if len(small_list) < chunk_size: #Keep adding prices until we reach chunk size
        small_list.append(val)  
               
    if size_flag == 'reached_chunk_size':        
        if small_list[-1] != val: 
            small_list.popleft()   #Remove the left most (first) price
            small_list.append(val) #Add current value to right most position

                
        change_position -= 1  #Price allocation in small list happens right to left so change position goes from chunk size to zero
        if change_position < 0: 
            change_position = chunk_size - 1 #If position reaches zero, it needs to be reset to chunk size - 1 (for index position)

            
    if len(small_list) == chunk_size: #check if small list has reached chunk size
        size_flag = 'reached_chunk_size'  
    
    return(small_list)

In [9]:
# Identify directions
def find_direction(lst):
    list_size = len(lst)
    diff = {}
    dir_flag = 'Error : not calculated'
   
    for i in range(list_size-1): #write the list into a dictionary 
        diff[i] = lst[i+1] - lst[i]       
        
    if all(x==0 for x in diff.values()):
        dir_flag = 'flat'
    
    elif all(x>0 for x in diff.values()):
        dir_flag = 'positive'
    
    elif all(x<0 for x in diff.values()):
        dir_flag = 'negative'
    
    elif list(diff.values())[-1] == 0 and list(diff.values())[-2] > 0:
        dir_flag = 'flat'

    elif list(diff.values())[-1] == 0 and list(diff.values())[-2] < 0:
        dir_flag = 'flat'

    elif list(diff.values())[-2] >= 0 and list(diff.values())[-1] < 0:
        dir_flag = 'dir_change_negative'      

    elif list(diff.values())[-2] <= 0 and list(diff.values())[-1] > 0:
        dir_flag = 'dir_change_positive'      
        
    else:
        dir_flag = 'no_single_direction'
        
    return(dir_flag)

In [10]:
full_list = [1,2,3,2,5,6,7,8,9,10,11,12,12,12,11,10,10,9,9,8,8,7,7]

chunk_size = 3
change_position = chunk_size - 1
small_list = collections.deque([])
size_flag = 0

for i, val in enumerate(full_list):    
    small_list = get_chunks1(i, val)
    if len(small_list) >= chunk_size:
        d = find_direction(small_list) 
        print(small_list,val, d)
    else:
        print(val)

1
2
deque([1, 2, 3]) 3 positive
deque([2, 3, 2]) 2 dir_change_negative
deque([3, 2, 5]) 5 dir_change_positive
deque([2, 5, 6]) 6 positive
deque([5, 6, 7]) 7 positive
deque([6, 7, 8]) 8 positive
deque([7, 8, 9]) 9 positive
deque([8, 9, 10]) 10 positive
deque([9, 10, 11]) 11 positive
deque([10, 11, 12]) 12 positive
deque([10, 11, 12]) 12 positive
deque([10, 11, 12]) 12 positive
deque([11, 12, 11]) 11 dir_change_negative
deque([12, 11, 10]) 10 negative
deque([12, 11, 10]) 10 negative
deque([11, 10, 9]) 9 negative
deque([11, 10, 9]) 9 negative
deque([10, 9, 8]) 8 negative
deque([10, 9, 8]) 8 negative
deque([9, 8, 7]) 7 negative
deque([9, 8, 7]) 7 negative


In [12]:
full_list = [1,2,3,2,5,6,7,8,9,10,11,12,12,12,11,10,10,9,9,8,8,7,7]

chunk_size = 9
change_position = chunk_size - 1
small_list = collections.deque([])
size_flag = 0

for i, val in enumerate(full_list):    
    small_list = get_chunks1(i, val)
    if len(small_list) >= chunk_size:
        d = find_direction(small_list) 
        print(small_list,val, d)
    else:
        print(val)

1
2
3
2
5
6
7
8
deque([1, 2, 3, 2, 5, 6, 7, 8, 9]) 9 no_single_direction
deque([2, 3, 2, 5, 6, 7, 8, 9, 10]) 10 no_single_direction
deque([3, 2, 5, 6, 7, 8, 9, 10, 11]) 11 no_single_direction
deque([2, 5, 6, 7, 8, 9, 10, 11, 12]) 12 positive
deque([2, 5, 6, 7, 8, 9, 10, 11, 12]) 12 positive
deque([2, 5, 6, 7, 8, 9, 10, 11, 12]) 12 positive
deque([5, 6, 7, 8, 9, 10, 11, 12, 11]) 11 dir_change_negative
deque([6, 7, 8, 9, 10, 11, 12, 11, 10]) 10 no_single_direction
deque([6, 7, 8, 9, 10, 11, 12, 11, 10]) 10 no_single_direction
deque([7, 8, 9, 10, 11, 12, 11, 10, 9]) 9 no_single_direction
deque([7, 8, 9, 10, 11, 12, 11, 10, 9]) 9 no_single_direction
deque([8, 9, 10, 11, 12, 11, 10, 9, 8]) 8 no_single_direction
deque([8, 9, 10, 11, 12, 11, 10, 9, 8]) 8 no_single_direction
deque([9, 10, 11, 12, 11, 10, 9, 8, 7]) 7 no_single_direction
deque([9, 10, 11, 12, 11, 10, 9, 8, 7]) 7 no_single_direction


In [ ]:
params = {
    'instruments': 'EUR_USD'
}
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

num = 4
bp_list = []
sp_list = []

for i, resp in enumerate(rv):
    if i < num:        
        print('Iteration : ',i)
        
        resp_type = resp['type']        
        
        if resp_type == 'PRICE':    
            date_val, time_val, time_fraction = get_date_time(resp)
            sell_price, buy_price, spread = get_prices(resp)          
            print('buy_price:',buy_price,',' , 'sell_price:',sell_price, ',', 'spread(pips):',spread*10000)
            print('----------------------------------------------')
            
        else:
            print('HEARTBEAT')

    else:
        terminate_connection()
        
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

# 2. fit into full code

In [ ]:
params = {
          "instrument": "EUR_USD",
          "time": "2020-06-06T04:00:00Z"
        }
        
r = instruments.InstrumentsPositionBook(instrument="EUR_USD", params=params)
temp = api.request(r)
hist_price_list = temp["positionBook"]["buckets"]
print(len(hist_price_list))
temp

In [ ]:
params = {
          "instrument": "EUR_USD",
          "time": "2020-06-05T05:00:00Z"
        }
        
r = instruments.InstrumentsPositionBook(instrument="EUR_USD", params=params)
temp = api.request(r)
hist_price_list = temp["positionBook"]["buckets"]
print(len(hist_price_list))
temp

In [ ]:
params = {
          "from": "2020-05-01T00:00:00Z",
          "to": "2020-05-01T01:00:00Z",
          "granularity": "S5"}

r = instruments.InstrumentsCandles(instrument="EUR_USD",
                                   params=params)

api.request(r)

In [ ]:
df = pd.read_csv('DAT_NT_EURUSD_T_BID_202005.csv', sep = ';', header = None, names = ['timestamp', 'price', 'unknown'])

In [ ]:
df.head()